In [1]:
import os
import subprocess


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import json 
from itertools import combinations

import pandas as pd
from generated_text_detector.utils.model.roberta_classifier import RobertaClassifier
from transformers import AutoTokenizer, AdamW
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader


/home/bussotti/.conda/envs/ForLLMLora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

f=open('list_gen_gpt_400.json')
tmp=json.load(f)
f.close()
obj=[]
for elt in tmp:
    obj+=[elt.lower().replace(',','').replace('.','').replace('?','').replace(';','').replace('!','').replace('\n','').replace('\'','')]
del tmp
obj={'rewrite':obj[:400],'merge':obj[400:800],'generate':obj[800:]}

In [3]:
# Load a parquet file into a pandas DataFrame
df_before = pd.read_parquet('before_gpt.parquet')

# Display the first few rows of the dataframe
# print(df_before.head())

txt_emails_to_use=list(df_before['cleaned_text'][:1200])

txt_emails_to_test=list(df_before['cleaned_text'][1200:1400])

In [4]:

items = list(obj.keys())

# Get all combinations of different lengths
all_combinations = []
for r in range(1, len(items) + 1):
    all_combinations.extend(combinations(items, r))

to_implement=[[]]
for combo in all_combinations:
    to_implement+=[list(combo)]


In [7]:
to_implement

[[],
 ['rewrite'],
 ['merge'],
 ['generate'],
 ['rewrite', 'merge'],
 ['rewrite', 'generate'],
 ['merge', 'generate'],
 ['rewrite', 'merge', 'generate']]

In [8]:


# Custom Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True,
            return_tensors="pt"
        )
        input_ids = inputs["input_ids"].squeeze(0)
        attention_mask = inputs["attention_mask"].squeeze(0)
        label = torch.tensor(self.labels[idx], dtype=torch.float)
        return {"input_ids": input_ids, "attention_mask": attention_mask, "label": label}


In [9]:
import torch

def evaluate_model(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    device = next(model.parameters()).device  # Ensure the model is on the correct device
    scores = []

    with torch.no_grad():  # No gradients needed for evaluation
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            logits = model(input_ids=input_ids, attention_mask=attention_mask)[1]

            # Convert logits to probabilities
            probabilities = torch.sigmoid(logits).squeeze(-1)

            # Append probabilities (scores) to list
            scores.extend(probabilities.cpu().numpy())  # Move to CPU and convert to numpy for easy handling

    # Calculate the average score across all test elements
    average_score = sum(scores) / len(scores) if scores else 0
    return average_score


In [10]:
from tqdm import tqdm

In [11]:
###########FROM GPT_ VARIANT051124


from transformers import RobertaForSequenceClassification, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset



for strategy in to_implement:
    if len(strategy)<3:
        continue
    name_folder='strategy'+'_'.join(strategy)
    save_directory = name_folder+"/fine_tuned_roberta_VARIANT061124_WITHPROB_TEMP2"
    if os.path.exists(save_directory) or len(strategy)==0:
        print('model already trained')
        continue
    if not os.path.exists(name_folder):
        os.mkdir(name_folder)
    train_texts=[]
    train_labels=[]
    for key in strategy:
        train_texts+=obj[key]
        train_labels+=[1 for x in obj[key]]
    train_texts+=txt_emails_to_use[:len(train_texts)]
    train_labels+=[0 for x in train_labels]

    model = RobertaClassifier.from_pretrained("SuperAnnotate/roberta-large-llm-content-detector")
    tokenizer = AutoTokenizer.from_pretrained("SuperAnnotate/roberta-large-llm-content-detector")

    model.roberta.config.hidden_dropout_prob = 0.2
    
    # Data preparation
    test_dataset = TextDataset(txt_emails_to_test, [0 for x in txt_emails_to_test], tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

    
    # Data preparation
    train_dataset = TextDataset(train_texts, train_labels, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    
    # Fine-tuning setup
    optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01, betas=(0.9, 0.98))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Scheduler setup
    num_epochs = 3
    accumulation_steps = 8  # Adjust this based on desired effective batch size
    num_training_steps = num_epochs * len(train_loader) // accumulation_steps
    warmup_steps = num_training_steps // 10  # 10% of total steps as warm-up
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)
    evaluate_every=500
    # Training loop with gradient accumulation and scheduler
    print(strategy)
    for epoch in range(num_epochs):
        total_loss = 0
        model.train()
        for i, batch in tqdm(enumerate(train_loader)):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
    
            # Forward pass
          
            logits = model(input_ids=input_ids, attention_mask=attention_mask)[1]  


            ######WITHOUT PROB
            
            #######WITH PROB
            temperature = 1.0  # Adjust as needed; higher values give smoother probabilities
            probabilities = torch.sigmoid(logits.squeeze(-1) / temperature)

            # Loss calculation
            ######WITHOUT PROB
            #loss = F.binary_cross_entropy_with_logits(logits.squeeze(-1), labels)
            #######WITH PROB
            loss = F.binary_cross_entropy_with_logits(probabilities, labels)

            
            loss = loss / accumulation_steps  # Scale loss for gradient accumulation
            loss.backward()
    
            # Accumulate gradients and update every `accumulation_steps`
            if (i + 1) % accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
                optimizer.step()
                scheduler.step()  # Update learning rate
                optimizer.zero_grad()  # Reset gradients
    
            total_loss += loss.item() * accumulation_steps  # Accumulate actual loss for logging
            if i%evaluate_every==0:
                # Assuming `test_loader` is defined
                average_score = evaluate_model(model, test_loader)
                print("Average Score on Test Set:", average_score)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")
    
    # Save the model and tokenizer
    model.save_pretrained(save_directory)
    tokenizer.save_pretrained(save_directory)


/home/bussotti/.conda/envs/ForLLMLora/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/bussotti/.conda/envs/ForLLMLora/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


['rewrite', 'merge', 'generate']


1it [00:10, 10.27s/it]

Average Score on Test Set: 0.21149490237236024


501it [03:50,  2.99s/it]

...

Average Score on Test Set: 0.00012868218091171


501it [03:48,  3.00s/it]

Average Score on Test Set: 0.00011664028126688208


600it [04:31,  2.21it/s]


Epoch 3/3, Loss: 0.508213345358769


In [10]:
strategy

['rewrite', 'merge', 'generate']

In [11]:
to_implement

[[],
 ['rewrite'],
 ['merge'],
 ['generate'],
 ['rewrite', 'merge'],
 ['rewrite', 'generate'],
 ['merge', 'generate'],
 ['rewrite', 'merge', 'generate']]